In [15]:
import requests
import json
import pickle
import pandas as pd
from time import sleep
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
from sklearn.metrics.pairwise import cosine_distances
from collections import defaultdict
import numpy as np


In [2]:
df_vid = pd.read_csv('ods_videos.csv')

In [4]:
df_vid.head(5)

,youtube_id,channel,title,publishedAt,description,url
0,46nnbxEZNw8,Компьютерные науки,068. Как извлечь из сырых данных пользу для му...,2018-07-03T15:05:30.000Z,В рамках конференции «Data&Science: маркетинг»...,https://www.youtube.com/watch?v=46nnbxEZNw8
1,RnUcRYAbWtY,Компьютерные науки,001. Прикладные задачи оптимизации – М.В. Бацын,2015-04-10T12:07:26.000Z,NaN,https://www.youtube.com/watch?v=RnUcRYAbWtY
2,frLW8UVp_Ik,Компьютерные науки,005. Тематическое моделирование на пути к разв...,2016-09-27T13:08:21.000Z,NaN,https://www.youtube.com/watch?v=frLW8UVp_Ik
3,7lerRDCZrhU,Компьютерные науки,Запись тренировки ML 29.07.2017,2017-07-29T10:48:27.000Z,Описание:\nРим Шаяхметов — Предсказание сердеч...,https://www.youtube.com/watch?v=7lerRDCZrhU
4,Zco55S3RNU0,Компьютерные науки,058. Круглый стол «Как анализ данных меняет м...,2018-02-21T12:11:45.000Z,В рамках конференции Яндекса «Большие данные в...,https://www.youtube.com/watch?v=Zco55S3RNU0


In [11]:
id2title = {idx:title for idx, title in df_vid[['youtube_id', 'title']].values}


In [5]:
id2trans = pickle.load(open('id2transcripts.pkl', 'rb'))

In [10]:
id2trans['2c7HGBuXOZ8'][:30]

[{'duration': 3.38, 'start': 6.87, 'text': 'такси недавно запустили серию'},
 {'duration': 5.15, 'start': 9.35, 'text': 'мероприятий д.т.н.'},
 {'duration': 5.9, 'start': 10.25, 'text': 'сайнс который людей от науки ра'},
 {'duration': 4.2,
  'start': 14.5,
  'text': 'про свои задачи которых анализ данных'},
 {'duration': 4.44,
  'start': 16.15,
  'text': 'мог бы помочь а людей развивается да и'},
 {'duration': 4.22, 'start': 18.7, 'text': 'касается могли бы узнать новую'},
 {'duration': 7.32,
  'start': 20.59,
  'text': 'предметную область для себя и в ней вот'},
 {'duration': 8.56,
  'start': 22.92,
  'text': 'покопаться и первый метод был про физику'},
 {'duration': 5.219,
  'start': 27.91,
  'text': 'высоко физика высоких энергий и в'},
 {'duration': 8.42,
  'start': 31.48,
  'text': 'частности на этом do it when silence of'},
 {'duration': 11.731,
  'start': 33.129,
  'text': 'митапе было выдано задание про которая'},
 {'duration': 10.96,
  'start': 39.9,
  'text': 'богдан который

In [18]:
index2id = {i:idx for i, (idx, _, _) in enumerate(pieces)}


In [12]:
# склеиваем кусочки транскриптов в куски длинной 30 сек
pieces = []

for key in id2trans:
    text = []
    duration = 0
    start = None
    for piece in id2trans[key]:
        if not piece['text']:
            continue
        text.append(piece['text'])
        duration += piece['duration']
        if not start:
            start = piece['start']
        
        if duration > 30:
            pieces.append((key, ' '.join(text), start, ))
            text = []
            duration = 0
            start = None



In [13]:
pieces[:3]

[('2c7HGBuXOZ8',
  'такси недавно запустили серию мероприятий д.т.н. сайнс который людей от науки ра про свои задачи которых анализ данных мог бы помочь а людей развивается да и касается могли бы узнать новую предметную область для себя и в ней вот',
  6.87),
 ('2c7HGBuXOZ8',
  'покопаться и первый метод был про физику высоко физика высоких энергий и в частности на этом do it when silence of митапе было выдано задание про которая',
  22.92),
 ('2c7HGBuXOZ8',
  'богдан который сделал лучшее решение сейчас расскажут всем привет меня зовут богдан я часто в этом конкурсе и за',
  39.9)]

In [23]:
texts = [p[1] for p in pieces]

Строим тфидф по транскриптам

In [24]:
tfidf = TfidfVectorizer(min_df=5,max_features=25000, ngram_range=(1,2))
# svd = TruncatedSVD(200)

index = tfidf.fit_transform(texts)

Транформим какой-то текст в вектор и считаем косинус до остальных транскриптов. Аггрегируем близости по ид_видео 

In [25]:
accum = defaultdict(list)

sims = cosine_distances(tfidf.transform(['стекинг']), index)[0]

for arg, sim in enumerate(sims):
    accum[index2id[arg]].append((arg, sim))

Находим наиболее близкий видос по среднему косинуса между нашим текстом и транскриптами этого видоса
Находим для видоса наиболее близкий траскрипт

In [26]:
id2mean = [(key, np.min([x[1] for x in elements])) for key, elements in accum.items()]
id2maxind = {key:min(elements, key=lambda x: x[1])[0] for key, elements in accum.items()}

In [27]:
id2mean_s = sorted(id2mean, key=lambda x: x[1])

Показываем самые близкие

In [30]:
[(id2title[idx], texts[id2maxind[idx]]) for idx, _ in id2mean_s[:5]]

[('016. Фильтрация событий Большого Адронного Коллайдера — Богдан Мельник, Андрей Устюжанин',
  'простая диета используя стекинг то есть я разбился данные по индикатору события на 5 фунтов обучил на них наивное решение получил для каждого fulda предсказания для'),
 ('Data&Science: Фильтрация событий Большого Адронного Коллайдера — Б. Мельник, А. Устюжанин',
  'простая диета используя стекинг то есть я разбился данные по индикатору события на 5 фунтов обучил на них наивное решение получил для каждого fulda предсказания для'),
 ('Sberbank Data Science Contest: предсказание трат клиентов — Дмитрий Алтухов',
  'линейности поймал и поэтому дальше какой-то тренинг или стекинг уже особо сильно не мог помочь потому что как бы все что можно мы уже забили и фичами и'),
 ('028. Sberbank Data Science Contest (предсказание трат клиентов) — Дмитрий Алтухов',
  'линейности поймал и поэтому дальше какой-то тренинг или стекинг уже особо сильно не мог помочь потому что как бы все что можно мы уже заб